<a href="https://colab.research.google.com/github/YousefIbrahem/ml_project/blob/main/setfit_example_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%reset -f

In [ ]:
import gc
gc.collect()

0

# Install required packages

In [ ]:
!pip install -q setfit datasets scikit-learn sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.5/75.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 87.2 MB/s eta 0:00:00


# Import libraries

In [ ]:
from setfit import SetFitModel, SetFitTrainer
from datasets import load_dataset
from sklearn.metrics import accuracy_score
from sentence_transformers.losses import CosineSimilarityLoss
from datasets import Dataset

# Load sentiment dataset

In [ ]:
dataset = load_dataset("sst2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/872 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [ ]:
train_dataset=dataset["train"].shuffle(seed=42).select(range(10))
train_data=[{"text":example["sentence"] ,
                "label":int(example["label"])} for example in train_dataset]
eval_dataset=dataset["validation"].select(range(8))
eval_data= [{"text":example["sentence"] ,
                "label":int(example["label"])} for example in eval_dataset]

In [ ]:
train_dataset =Dataset.from_list(train_data)
eval_dataset = Dataset.from_list(eval_data)

In [ ]:
dataset= dataset.rename_column ("sentence" , "text")

In [ ]:
dataset= dataset.remove_columns ("idx")

In [ ]:
print (dataset["train"].column_names)

['text', 'label']


# Load pretrained model

In [ ]:
model = SetFitModel.from_pretrained("sentence-transformers/paraphrase-MiniLM-L3-v2")

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

model_head.pkl not found on HuggingFace Hub, initialising classification head with random weights. You should TRAIN this model on a downstream task to use it for predictions and inference.


# Set up trainer

In [ ]:
trainer = SetFitTrainer(
    model=model,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"] ,
    loss_class= CosineSimilarityLoss ,
    batch_size=4,
    num_iterations=3,
    num_epochs=1,
    metric=accuracy_score,

)


/tmp/ipython-input-2930420448.py:1: DeprecationWarning: `SetFitTrainer` has been deprecated and will be removed in v2.0.0 of SetFit. Please use `Trainer` instead.
  trainer = SetFitTrainer(


AttributeError: 'Column' object has no attribute 'sort'

# Train the model

In [ ]:
trainer.train()

# Evaluate the model

In [ ]:
accuracy = trainer.evaluate()
print("Model accuracy:", accuracy)


# Predict on new texts

In [ ]:
examples = ["I love this movie!", "This is the worst day ever."]
predictions = trainer.model.predict(examples)
